# ユーザーをグループに追加します

ユーザーをグループに追加します。JupyterHub上にユーザーが作成済みである必要があります。

所属グループに応じたリソース制限設定に使用します。

# グループの定義

グループ名

In [ ]:
group_name = 'group1'
group_name

このグループに所属させるユーザーのメールアドレス

In [ ]:
user_email_list = '''
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''

In [ ]:
import re
user_emails = user_email_list.splitlines()
user_emails = [x.strip() for x in user_emails 
               if re.sub(r'#.*$', '', x).strip()]
user_emails

In [ ]:
import hashlib

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

user_names = [get_username_from_mail_address(x) for x in user_emails]
user_names

# グループの設定

In [ ]:
import os
apitoken = os.environ['JUPYTERHUB_API_TOKEN']
api_url = os.environ['JUPYTERHUB_API_URL']

グループの作成（必要な場合）

In [ ]:
headers = {
    'Authorization': 'token %s' % apitoken,
}

In [ ]:
import requests
import json

r = requests.get(api_url + '/groups/{}'.format(group_name),
                 headers=headers)
if r.status_code == 404:
    r = requests.post(api_url + '/groups/{}'.format(group_name),
                      headers=headers)
    group = json.loads(r.text)
else:
    group = json.loads(r.text)

group

In [ ]:
import json

r = requests.get(api_url + '/groups/{}'.format(group_name),
                 headers=headers)
group = json.loads(r.text)
current_users = group['users']

user2mail = dict(zip(user_names, user_emails))

new_users =  set(user_names) - set(current_users)

if new_users:
    print("Add the following users to group '{}'".format(group_name))
    for username in new_users:
        print('{}({})'.format(user2mail[username], username))
    h = headers.copy()
    h['Content-Type'] : 'application/json'
    r = requests.post(api_url + '/groups/{}/users'.format(group_name),
                      data = json.dumps({
                          "users" : list(new_users)
                      }),
                      headers=h)
    assert r.status_code == 200
member_users = set(user_names) - set(new_users)
if member_users:
    print("The fowllowing users are already member of group '{}'".format(group_name))
    for username in member_users:
        print('{}({})'.format(user2mail[username], username))


確認

In [ ]:
r = requests.get(api_url + '/groups/{}'.format(group_name),
                  headers=headers)
group = json.loads(r.text)
group